In [1]:
# This code does transformations on each dataset to clean fields. Transformations added to:
# 1. salary -> remove comma and convert to int
# 2. location -> city and state
# 3. dates -> convert to datetime and create year
# 4. greencard, h1b visa status -> remove records with status of WITHDRAWN or INVALIDATED

In [2]:
import datetime
import pandas as pd
import re

In [3]:
directory = '/Users/kwheatley/Desktop/Capstone/gcloud_data/'

In [4]:
# Load the dictionary of state abbreviations to state names
state_abbr_dict = pd.read_csv("functions/configuration_files/state_abbr_dict.csv")
state_abbr_dict = state_abbr_dict.set_index('abbreviation')['result_state'].to_dict()

# Process H1B Salaries

In [14]:
filename = 'salaries_h1b'
data = pd.read_csv(directory+"01_"+filename+".csv")

# There are 4 statuses available: 'CERTIFIED', 'WITHDRAWN', 'DENIED', 'INVALIDATED'
# Let's remove all records with WITHDRAWN or INVALIDATED
# We don't know the details of why they were removed
data = data[data.status.isin(['CERTIFIED', 'DENIED'])]

# Convert salary data into int
data['salary'] = data.salary.apply(lambda x: x.replace(',','')).astype(int)

# Poverty line is $12,486; let's remove all the salaries less than $15k
data = data[data.salary >= 15000]

# Split location to city and state
data['city'] = data.location.apply(lambda x: x.lower().split(',')[0])
data['state'] = data.location.apply(lambda x: x.lower().split(',')[1])

# Clean city and state data; remove numbers and special characters
data['city'] = data.city.apply(lambda x: re.sub('[^ A-Za-z]+', '', x).strip())
data['state'] = data.state.apply(lambda x: re.sub('[^ A-Za-z]+', '', x).strip())

# Replace state abbreviation with cleaned state name
data['state'] = data.state.apply(lambda x: state_abbr_dict[x] if x in state_abbr_dict else 'Unknown')

# Convert dates into datetime format and pull out years
data['submit_date'] = data.submit_date.apply(lambda x: datetime.datetime.strptime(x, '%M/%d/%Y'))
data['start_date'] = data.start_date.apply(lambda x: datetime.datetime.strptime(x, '%M/%d/%Y'))
data['submit_year'] = data.submit_date.apply(lambda x: str(x.year))
data['start_year'] = data.start_date.apply(lambda x: str(x.year))

data.to_csv(directory+"02_"+filename+".csv",index=False)

# Process Greencard Salaries

In [15]:
filename = 'salaries_greencard'
data = pd.read_csv(directory+"01_"+filename+".csv")

# There are 4 statuses available: 'Certified', 'Denied', 'Withdrawn', 'Certified-expired', 'Certified-Expired'
# Let's remove all records with Withdrawn
# We don't know the details of why they were removed
data = data[data.status.isin(['Certified', 'Denied', 'Withdrawn', 'Certified-expired', 'Certified-Expired'])]

# Clean salary information; split amount from type
data['salary_amount'] = data.salary.apply(lambda x: x.split('/')[0].strip())
data['salary_type'] = data.salary.apply(lambda x: x.split('/')[1].lower().strip())

# Convert salary amount into float
data['salary_amount'] = data.salary_amount.apply(lambda x: x.replace('#','').replace(',','').strip())
data = data[data.salary_amount != '']
data['salary_amount'] = data.salary_amount.astype(float)

# Here are all the possible salary types: 
# 'hr', 'yr', 'wk', 'bi', 'mth', 'year', 'hour', '', 'month', 'week','bi-weekly'
# We will calculate salary based on each type
list_salaries = []
for index, row in data[['salary_amount','salary_type']].iterrows():
    if row.salary_type in ('hr','hour','wk','week','bi','bi-weekly','yr','year','') \
        and row.salary_amount >= 15000:
            calc_salary = row.salary_amount
    else: calc_salary = None
    list_salaries.append(calc_salary)
data.salary_amount = list_salaries

# Split location into city and state
data.location = data.location.apply(lambda x: x.replace(',,',','))
data['city'] = data.location.apply(lambda x: x.lower().split(',')[0] if ',' in x else '')
data['state'] = data.location.apply(lambda x: x.lower().split(',')[1] if ',' in x else x)

# Clean city and state data; remove numbers and special characters
data['city'] = data.city.apply(lambda x: re.sub('[^ A-Za-z]+', '', x).strip())
data['state'] = data.state.apply(lambda x: re.sub('[^ A-Za-z]+', '', x).strip())

# Replace state abbreviation with cleaned state name
data['state'] = data.state.apply(lambda x: state_abbr_dict[x] if x in state_abbr_dict else 'Unknown')

# Convert dates into datetime format and pull out years
data.decision_date = data.decision_date.apply(lambda x:'20'+x.split('/')[2] +'-'+ \
                                    ('0'+x.split('/')[0])[-2:] +'-'+ \
                                    ('0'+x.split('/')[1])[-2:] if '/' in x else x)
data['decision_date'] = data.decision_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M-%d'))
data['decision_year'] = data.decision_date.apply(lambda x: str(x.year))

data.to_csv(directory+"02_"+filename+".csv",index=False)

# Process Indeed Resumes

In [13]:
filename = 'resumes_work'
data = pd.read_csv(directory+"01_"+filename+".csv")

# Split location into city and state
data.location = data.location.fillna("")
data.location = data.location.apply(lambda x: x.replace(',,',','))
data['city'] = data.location.apply(lambda x: x.split(',')[0].strip() if ',' in x else '')
data['state'] = data.location.apply(lambda x: x.split(',')[1].strip() if ',' in x else x)

# Split date information
# If no end date, then use December in the start year as the end date
# If "Present" for end date, then use June 2018 as the end date
data['from_date'] = data.dates.apply(lambda x: (x.split(" to ")[0]).strip())
data['from_date'] = data.from_date.apply(lambda x: 
                        datetime.datetime.strptime('January ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['to_date'] = data.dates.apply(lambda x: (x.split(" to ")[1]).strip()
                        if len(x.split(" to "))>1
                        else x.strip())
data.loc[data.to_date == 'Present','to_date'] = 'July 2018'
data['to_date'] = data.to_date.apply(lambda x: 
                        datetime.datetime.strptime('December ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['from_year'] = data.from_date.apply(lambda x: str(x.year))
data['to_year'] = data.to_date.apply(lambda x: str(x.year))
data['days_worked'] = (data.to_date - data.from_date).dt.days
data['years_worked'] = round(data.days_worked/365)

data.to_csv(directory+"02_"+filename+".csv",index=False)

In [18]:
filename = 'resumes_education'
data = pd.read_csv(directory+"00_"+filename+".csv",header=None)
data.columns = ['search_city','resume_id','container_type','edu_title',
                      'edu_school','edu_addressLocality','edu_dates']

# Remove any null education titles
data = data[~data.edu_title.isnull()]

data['city'] = data.edu_addressLocality.apply(
                            lambda x: x.lower().split(',')[0]
                            if len(str(x).split(','))>1
                            else x)
data['state'] = data.edu_addressLocality.apply(
                            lambda x: x.lower().split(',')[1] 
                            if len(str(x).split(','))>1 and str(x)!='nan'
                            else x)
data['state'] = data.state.apply(lambda x: state_abbr_dict[x] if x in state_abbr_dict else x)

# Update dates
data.loc[data.edu_dates.isnull(),'edu_dates'] = 'January 1900'
data['from_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[0]).strip())
data['from_date'] = data.from_date.apply(lambda x: 
                        datetime.datetime.strptime('January ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['to_date'] = data.edu_dates.apply(lambda x: (x.split(" to ")[1]).strip()
                        if len(x.split(" to "))>1
                        else x.strip())
data['currently_here'] = 'No'
data.loc[data.to_date == 'Present','currently_here'] = 'Yes'
data.loc[data.to_date == 'Present','to_date'] = 'July 2018'
data['to_date'] = data.to_date.apply(lambda x: 
                        datetime.datetime.strptime('December ' + x, '%B %Y')
                        if len(x.split(" "))==1
                        else datetime.datetime.strptime(x, '%B %Y'))
data['from_year'] = data.from_date.apply(lambda x: str(x.year))
data['to_year'] = data.to_date.apply(lambda x: str(x.year))
data['days_edu'] = (data.to_date - data.from_date).dt.days
data['years_edu'] = round(data.days_edu/365)

data.to_csv(directory+"02_"+filename+".csv",index=False)

# Process Indeed Job Postings

In [19]:
filename = 'job_posts_indeed'
data = pd.read_csv(directory+"01_"+filename+".csv")

# Split the city, state, and zip 
data.location = data.location.fillna("")
data.location = data.location.apply(lambda x: x.replace(',,',','))
data['city'] = data.location.apply(lambda x: x.split(',')[0].strip() if ',' in x else '')
data['state'] = data.location.apply(lambda x: x.split(',')[1].strip() if ',' in x else x)
data['zip'] = data.state.apply(lambda x: x.split(' ')[1].strip() if any(char.isdigit() for char in x) else '')
data['state'] = data.state.apply(lambda x: x.split(' ')[0].strip() if any(char.isdigit() for char in x) else x)

# I have not done anything to convert the dates. Sometimes there is not an accurate date for Indeed job postings.
# We will ignore dates anyway for Indeed Job postings.

data.to_csv(directory+"02_"+filename+".csv",index=False)

# Process CCAR Job Postings¶

In [20]:
filename = 'job_posts_ccars'
data = pd.read_csv(directory+"01_"+filename+".csv")

# Convert date field to datetime format
data['datePosted'] = data.datePosted.apply(lambda x: datetime.datetime.strptime(x, '%Y-%M-%d'))
data['datePosted_year'] = data.datePosted.apply(lambda x: str(x.year))

# Change names for city and state
data['state'] = data.region
data['city'] = data.locality
del data['region']
del data['locality']

data.to_csv(directory+"02_"+filename+".csv",index=False)

# End